### Ingestión de carpeta "production_company"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer archvivos CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
productions_companies_schema = StructType(fields= [
    StructField("companyId", IntegerType(), True),
    StructField("companyName", StringType(), True)
])                       

In [0]:
productions_companies_df = spark.read \
        .schema(productions_companies_schema) \
        .csv(f"{bronze_folder_path}/{v_file_date}/production_company") 
        #.csv("/mnt/moviehistoryjaag/bronze/production_company/production_company_*.csv") 

#### Pase 2 - Renombrar las columnas y añadir nuevas columnas
1. "companyId" renombrar a "company_id"
2. "companyName" renombrar a "company_name"
33. Agregar la columnas "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
productions_companies_final_df = add_ingestion_date(productions_companies_df) \
    .withColumnsRenamed({"companyId" : "company_id",
                         "companyName" : "company_name"}) \
    .withColumn("environment", lit(v_environment)) \
    .withColumn("file_date", lit(v_file_date))    


#### Paso 3 - Escribir datos en el DataLake en formato "Parquet"

In [0]:
#overwrite_partition(productions_companies_final_df, "movie_silver","productions_companies", "file_date")

In [0]:
merge_condition = "tgt.company_id = src.company_id AND tgt.file_date = src.file_date"
merge_delta_lake(productions_companies_final_df, "movie_silver","productions_companies", silver_folder_path, merge_condition, "file_date")
#productions_companies_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.productions_companies")
#productions_companies_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/productions_companies")

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.productions_companies
GROUP BY file_date;

In [0]:
%sql
SELECT * FROM movie_silver.productions_companies;

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/productions_companies"))

In [0]:
dbutils.notebook.exit("Success")